# From Text to a Knowledge Graph

Based on article: *[From Text to a Knowledge Graph. Hands-on](https://medium.com/@hugojm/from-text-to-a-knowledge-graph-hands-on-dd68e9d42939)*

Libraries: 
* [DBpedia Spotlight for SpaCy](https://spacy.io/universe/project/spacy-dbpedia-spotlight): **pip install spacy_dbpedia_spotlight**
* [Getting started with RDFLib](https://rdflib.readthedocs.io/en/stable/gettingstarted.html): **conda install -c conda-forge rdflib**

In [1]:
import spacy
import spacy_dbpedia_spotlight
from spacy.matcher import Matcher

UnsupportedOperation: fileno

In [ ]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('dbpedia_spotlight')

In [ ]:
doc = nlp("Joe Biden married Jill Tracy Jacobs, who was educated at the University of Delaware")

## PreProcessament 

### Tokenisation

In [ ]:
print([token.text for token in doc], sep=' ')

### Lemmatisation

In [ ]:
print([token.lemma_ for token in doc], sep=' ')

### Part-of-speech Tagging

In [ ]:
print([(token.lemma_, token.pos_) for token in doc], sep=' ')

### Dependency Parsing

In [ ]:
print([(token.lemma_, token.dep_) for token in doc], sep=' ')

In [ ]:
from spacy import displacy
displacy.render(doc, style="dep")

### Named Entity Recognition (NER)

In [ ]:
print([(ent.text, ent.label_) for ent in doc.ents], sep=' ')

In [ ]:
ents = [ent for ent in doc.ents]

In [ ]:
ents_t = [ent.text for ent in doc.ents]

In [ ]:
print("Before: ", [token.text for token in doc])

with doc.retokenize() as retokenizer:
    for i in range(len(ents)):
        retokenizer.merge(doc.ents[i])
print("After: ", [token.text for token in doc])

### Named Entity Linking (NEL)

In [ ]:
print([ent.kb_id_ for ent in doc.ents])

### Relation Extraction (RE)

In [ ]:
def get_relation(sent):
    aux = []
    doc = nlp(sent)
    
    # Matcher class object
    matcher = Matcher(nlp.vocab)
    
    # define the pattern
    pattern = [[{'DEP' : 'ROOT'}],[{'DEP':'RELCL'}], [{'POS': 'VERB'}]]
    
    matcher.add("matching_1", pattern)
    
    matches = matcher(doc)
    for x in matches:
        aux.append(doc[x[1]:x[2]].text)
        
    return(aux)

In [ ]:
rels = get_relation("Joe Biden married Jill Tracy Jacobs, who was educated at the University of Delaware")
print(rels)

## Building Knowledge Graph

Finally we are going to create the final Knowledge Graph with the information we have.

In [ ]:
from rdflib import Graph, URIRef, Literal, BNode
from rdflib.namespace import RDF, RDFS
from rdflib import Namespace

In [ ]:
g = Graph()

for ent in doc.ents:
    # add the label of the entry
    g.add((URIRef(ent.kb_id_), RDFS.label, Literal(ent.text)))
    prev = URIRef(ent.kb_id_)
    
g.add((URIRef('http://dbpedia.org/resource/Joe_Biden'), URIRef("https://dbpedia.org/ontology/"+rels[0]), URIRef('http://dbpedia.org/resource/Jill_Tracy'))) 
g.add((URIRef('http://dbpedia.org/resource/Jill_Tracy'), URIRef("https://dbpedia.org/ontology/"+rels[1]), URIRef('http://dbpedia.org/resource/Delaware')))

In [ ]:
for s,p, o in g:
    print(s,p,o)

## Visualizing the Knowledge Graph

In [ ]:
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
import matplotlib.pyplot as plt

G = rdflib_to_networkx_multidigraph(g)

#  Plot Networkx instance of RDF Graph
pos = nx.spring_layout(G, scale=2)
edge_labels = nx.get_edge_attributes(G, 'r')
plt.figure(figsize=(10,10))
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
nx.draw(G, with_labels=True)

plt.show()